In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\MY Laptop\Desktop\guvi_class\smart premium\train.csv")

In [2]:

df['Marital Status']=df[['Marital Status']].fillna(df['Marital Status'].mode()[0])
df['Age'] = df[['Age']].fillna(df['Age'].median())
df['Number of Dependents'] = df[['Number of Dependents']].fillna(df['Number of Dependents'].median())
df['Previous Claims'] = df[['Previous Claims']].fillna(df['Previous Claims'].median())
df['Occupation'] = df[['Occupation']].fillna(df['Occupation'].mode()[0])
df['Health Score'] = df[['Health Score']].fillna(df['Health Score'].median())
df['Credit Score'] = df[['Credit Score']].fillna(df['Credit Score'].median())
df['Customer Feedback'] = df[['Customer Feedback']].fillna(df['Customer Feedback'].mode()[0])
df['Vehicle Age'] = df[['Vehicle Age']].fillna(df['Vehicle Age'].median())
df['Insurance Duration'] = df[['Insurance Duration']].fillna(df['Insurance Duration'].median())
df['Annual Income'] = df[['Annual Income']].fillna(df['Annual Income'].median())

In [3]:
nominal_data=['Property Type','Marital Status','Gender','Occupation'] #OHC
ordinal_data=['Education Level', 'Location','Policy Type','Customer Feedback','Exercise Frequency','Smoking Status']
df['Customer Feedback']=df['Customer Feedback'].replace({'Poor':0,'Average':1 , 'Good':2})
df['Exercise Frequency']=df['Exercise Frequency'].replace({'Rarely':0,'Monthly':1 ,'Weekly':2 ,'Daily':3})
df['Location']=df['Location'].replace({'Rural':0,'Suburban':1 , 'Urban':2})
df['Education Level']=df['Education Level'].replace({'High School': 0,"Bachelor's":1 ,"Master's":2, 'PhD':3})
df['Smoking Status']=df['Smoking Status'].replace({'No':0,'Yes':1})
df['Policy Type']=df['Policy Type'].replace({'Basic':0,'Comprehensive':1,'Premium':2})
df=pd.get_dummies(df,columns=nominal_data,dtype='int')
df.drop(['Policy Start Date','id'],axis=1,inplace=True)


C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7340\1217611785.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Customer Feedback']=df['Customer Feedback'].replace({'Poor':0,'Average':1 , 'Good':2})
C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7340\1217611785.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Exercise Frequency']=df['Exercise Frequency'].replace({'Rarely':0,'Monthly':1 ,'Weekly':2 ,'Daily':3})
C:\Users\MY Laptop\AppData\Local\Temp\ipykernel_7340\1217611785.py:5: FutureWarning: Downcasting behavior 

In [4]:
import numpy as np
outliers=['Annual Income','Previous Claims', 'Premium Amount']
for col in outliers:

    Q1 = df[col].quantile(0.25)  # 25th percentile
    Q3 = df[col].quantile(0.75)  # 75th percentile
    IQR = Q3 - Q1  # Interquartile range

    # Define lower and upper bound
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap values at lower and upper bound
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

In [5]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
df['Annual Income'] = pt.fit_transform(df[['Annual Income']])

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import numpy as np
from xgboost import XGBRegressor
X=df.drop(['Premium Amount'],axis=1)
# X.isna().sum()
y=df['Premium Amount']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [ ]:
# single register model in mlflow
params={
    "n_estimators":25,
    "max_depth":10,
    "n_jobs":-1,
}
# import mlflow
mlflow.set_experiment('First exp')
mlflow.set_tracking_uri("http://127.0.0.1:5000")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric('r2',r2_score(y_test,test_pred))
    mlflow.sklearn.log_model(model,"Random Forest Regressor")

## multiple model register in mlflow 

In [7]:
import xgboost as xgb
models=[(
    "RandomForestRegressor",
    {"n_estimators":40,"max_depth":5,"n_jobs":-1},
    RandomForestRegressor(),
    (x_train,y_train),
    (x_test,y_test)
    
),
(
    "XGBRegressor",
    {'objective':'reg:squarederror',"n_estimators":50,"max_depth":5,"n_jobs":-1, "learning_rate" :0.1},
    xgb.XGBRegressor(),
    (x_train,y_train),
    (x_test,y_test)

),
(
    "LinearRegression",
    {},
    LinearRegression(),
    (x_train,y_train),
    (x_test,y_test)

),
(
    "DecisionTreeRegressor",
    {},
    DecisionTreeRegressor(),
    (x_train,y_train),
    (x_test,y_test)

)
]

In [9]:
import numpy as np
reports=[]
for model_name,params,model,train_set,test_set in models:
    x_train=train_set[0]
    y_train=train_set[1]
    x_test=test_set[0]
    y_test=test_set[1]

    model.set_params(**params)
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)

    rmse=np.sqrt(mean_squared_error(y_test, y_pred))
    mae=mean_absolute_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    reports.append((model_name,rmse,mae,r2))

In [15]:
import mlflow
mlflow.set_experiment('second experiment')
mlflow.set_tracking_uri("http://127.0.0.1:5000")
for i,element in  enumerate(models):
    model_name=element[0]
    params=element[1]
    model=element[2]
    report=reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metric('RMSE',report[1])
        mlflow.log_metric('MAE',report[2])
        mlflow.log_metric('R2',report[3])

        if 'XGBRegressor' in model_name:
             mlflow.xgboost.log_model(model,"model")
        else :
             mlflow.sklearn.log_model(model,"model")
    
    

2025/04/13 23:23:37 INFO mlflow.tracking.fluent: Experiment with name 'second experiment' does not exist. Creating a new experiment.
2025/04/13 23:29:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\MYLAPT~1\AppData\Local\Temp\tmpazuowe1f\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/13 23:29:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForestRegressor at: http://127.0.0.1:5000/#/experiments/565843742065028286/runs/61386c089fe14f5092500e4d5d738d8c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/565843742065028286


C:\Users\MY Laptop\AppData\Roaming\Python\Python312\site-packages\xgboost\sklearn.py:1028: UserWarning: [23:29:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/04/13 23:30:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBRegressor at: http://127.0.0.1:5000/#/experiments/565843742065028286/runs/dd6525c5a3594ca2ba5d10d375e72710
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/565843742065028286


2025/04/13 23:30:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LinearRegression at: http://127.0.0.1:5000/#/experiments/565843742065028286/runs/1eb17a290ae14c9cafd641091d124189
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/565843742065028286


2025/04/13 23:31:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTreeRegressor at: http://127.0.0.1:5000/#/experiments/565843742065028286/runs/d8b6d7042bae4bf9877558e010277c79
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/565843742065028286
